In [3]:
import os
import numpy as np
import joblib
import tensorflow as tf
import random

In [4]:
## 2. Criação de uma SEED

SEED = 0

def criar_seed(seed=SEED):
  os.environ['PYTHONHASHSEED'] = str(seed) # Controla aleatoriedade interna do interpretador Python
  random.seed(seed) # Controla aleatoriedade da biblioteca random
  np.random.seed(seed) # Controla aleatoriedade do NumPy
  tf.random.set_seed(seed) # Controla aleatoriedade do TensorFlow (pesos, embaralhamento, etc)

def setar_deterministico(seed=SEED): # Garantir que se obtenha os mesmos valores ao rodar o código novamente
  criar_seed(seed=seed) # Setar as seeds
  os.environ['TF_DETERMINISTIC_OPS'] = '1' # Força o TensorFlow a usar operações determinísticas
  os.environ['TF_CUDNN_DETERMINISTIC'] = '1' # Força operações determinísticas específicas da GPU (cuDNN)
  tf.config.threading.set_inter_op_parallelism_threads(1) # Limita o paralelismo entre operações a 1 thread
  tf.config.threading.set_intra_op_parallelism_threads(1) # Limita o paralelismo dentro de cada operação a 1 thread

setar_deterministico(SEED)  # Executa tudo acima com a seed escolhida

In [ ]:
# === usar dados salvos (sem baixar nada) ===
import os, numpy as np, joblib

# ajuste só esses dois conforme sua pasta/ativo
TICKER   = "BBDC4"
DATA_DIR = "../Dados/Treinamento/BBDC4"  # ex.: "../Dados/Treinamento/ABEV3" se estiver em outra pasta

# caminhos
pXtr = os.path.join(DATA_DIR, f"X_train_{TICKER}.npy")
pYtr = os.path.join(DATA_DIR, f"y_train_{TICKER}.npy")
pXte = os.path.join(DATA_DIR, f"X_test_{TICKER}.npy")
pYte = os.path.join(DATA_DIR, f"y_test_{TICKER}.npy")
pSc  = os.path.join(DATA_DIR, f"scaler_{TICKER}.pkl")

# (opcional) se você salvou os y reais já desnormalizados
pYtr_real = os.path.join(DATA_DIR, f"y_real_train_{TICKER}.npy")
pYte_real = os.path.join(DATA_DIR, f"y_real_{TICKER}.npy")

# carregamento
X_train = np.load(pXtr)
y_train = np.load(pYtr)
X_test  = np.load(pXte)
y_test  = np.load(pYte)

scaler  = joblib.load(pSc)

# shapes úteis
n_timesteps = X_train.shape[1]
n_features  = X_train.shape[2]

# garante compatibilidade scaler <-> features
if hasattr(scaler, "n_features_in_"):
    assert scaler.n_features_in_ == n_features, (
        f"Scaler foi treinado com {scaler.n_features_in_} features, "
        f"mas X_* tem {n_features}."
    )

print("OK: dados salvos carregados.")
print("X_train:", X_train.shape, "| y_train:", y_train.shape)
print("X_test :", X_test.shape,  "| y_test :", y_test.shape)
print("n_timesteps:", n_timesteps, "| n_features:", n_features)


In [ ]:
# === carrega y real desnormalizado se existir (facilita nos gráficos) ===
y_real_train = None
y_real_test  = None

if os.path.exists(pYtr_real):
    y_real_train = np.load(pYtr_real)
if os.path.exists(pYte_real):
    y_real_test  = np.load(pYte_real)

if y_real_train is not None and y_real_test is not None:
    print("OK: y_real_train/y_real_test carregados (desnormalizados).")
else:
    print("Aviso: y_real_* não encontrados. Vamos desfazer a normalização com o scaler após o predict.")